In [9]:
import cv2
import mediapipe as mp
import numpy as np
import xgboost as xgb
import math

dic_classes = {0: 'Abacaxi', 1: 'Acompanhar', 2: 'Acordar', 3: 'Acrescentar', 4: 'Alto'}

# Carregar o modelo XGBoost previamente treinado
model = xgb.Booster()
model.load_model('xgboost_5_classes.model')
# Definir o método da árvore para 'gpu_hist' para utilizar a GPU
param = {'tree_method': 'gpu_hist'}
# Atualizar os parâmetros do modelo
model.set_param(param)

# Inicializar o MediaPipe Holistic
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic()

# Inicializar a webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Erro ao capturar o frame")
        break

    # Detectar as coordenadas dos gestos com o MediaPipe Holistic
    results = holistic.process(frame)

    gesture_coords = []

    if results.pose_landmarks:
        for landmark_id, landmark in enumerate(results.pose_landmarks.landmark):
            gesture_coords.extend([landmark.x, landmark.y, landmark.z])
    else:
        gesture_coords.extend([math.nan] * 33 * 3)
        
    if results.left_hand_landmarks:
        for landmark_id, landmark in enumerate(results.left_hand_landmarks.landmark):
            gesture_coords.extend([landmark.x, landmark.y, landmark.z])
    else:
        gesture_coords.extend([math.nan] * 21 * 3) 

    if results.right_hand_landmarks:
        for landmark_id, landmark in enumerate(results.right_hand_landmarks.landmark):
            gesture_coords.extend([landmark.x, landmark.y, landmark.z])
    else:
        gesture_coords.extend([math.nan] * 21 * 3) 

    gesture_coords = np.array(gesture_coords).reshape(1, -1)
    prediction = model.predict(xgb.DMatrix(gesture_coords))[0]

    # Mostrar o resultado da predição
    if np.max(prediction) > 0.6:
        prediction = dic_classes[np.argmax(prediction)]
    else:
        prediction = 'Nenhum'

    cv2.putText(frame, f'Gesto: {prediction}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Mostrar o frame
    cv2.imshow('Webcam', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()
